## step-0-train-modelでトレーニングしたモデルに推論リクエストを投げる

このノートブックを実行する時のヒント：   
- このノートブックは大容量のRawデータを読み込むため、<span style="color: orange; ">メモリー8GB以上のインスタンス</span>で実行してください
- KernelはPython3（Data Science）で動作確認をしています。

In [2]:
import os
import boto3
import pandas as pd
import time
from datetime import datetime
import model_utils

In [3]:
def get_data_for_pred(target, sampling_rate):
    previous_year, previous_month = model_utils.get_previous_year_month(target.year, target.month)
    df_previous_month = model_utils.get_raw_data(previous_year, previous_month, sampling_rate)
    df_current_month = model_utils.get_raw_data(target.year, target.month, sampling_rate)
    df_data = pd.concat([df_previous_month, df_current_month])
    del df_previous_month
    del df_current_month

    # Extract features
    df_features = model_utils.extract_features(df_data)
    df_features = model_utils.filter_current_month(df_features, target.year, target.month)
    
    return df_features

In [4]:
# 推論に利用するエンドポイントの名前を設定する
# 同じモデルが複数の推論エンドポイントにデプロイされている場合に、テスト時間の短縮のためそれらを平行して利用することも可能。
useable_endopoints = ['nyctaxi-xgboost-endpoint']
# useable_endopoints = [
#     'nyctaxi-xgboost-endpoint-0', 
#     'nyctaxi-xgboost-endpoint-1', 
#     'nyctaxi-xgboost-endpoint-2', 
#     'nyctaxi-xgboost-endpoint-3', 
# ]

# 元データのサンプリングレート
# モデル学習時と同じサンプリングレートを設定する
sampling_rate = 20

# 2020/03/03から2022/08/04までを7日間飛ばしで推論する設定
# 1時間ごとに1日分の推論を実行し、hourly()のモニタリング設定であれば1時間に1日分のレポートが生成される
start = '2020-03-03'
end = '2022-08-04'
day_step = 7
freq=f'{day_step}D'

### 指定した期間の推論を実行する
モニタリング周期に合わせて推論を投入することで、1日分の推論をモニタリングジョブに与えることができる

In [ ]:
result_dir = 'prediction_results_model_quality'
if not os.path.exists(result_dir):
    os.makedirs(result_dir)


currently_cached_month = 99
results={}
for i, target_date in enumerate(pd.date_range(start=start, end=end, freq=freq)):
    # Load raw data and extract features for new month
    if target_date.month != currently_cached_month:
        print('Loading data for', target_date.strftime('%Y-%m'))
        df_features = get_data_for_pred(target_date, sampling_rate)
        currently_cached_month = target_date.month
        
    # Select next endpoint
    endpoint = useable_endopoints[i%len(useable_endopoints)]
    print('Predicting', target_date, endpoint)

    # Exec prediction for the target date
    df_pred = df_features[df_features.index == target_date].copy()
    df_pred[['pred', 'inference_id']] = model_utils.exec_prediction(endpoint, df_pred)
    df_pred.to_csv(f'{result_dir}/prediction-result-{target_date.strftime("%Y-%m-%d")}.csv', index=False)
    results[target_date.strftime('%Y-%m-%d')] = df_features

    # Wait for next report cycle
    if endpoint == useable_endopoints[-1]:
        sleep_sec_until_5mins = (65 - pd.to_datetime(datetime.now()).minute) * 60
        print('Sleep until next monitoring cycle. {} seconds\n'.format(sleep_sec_until_5mins))
        time.sleep(sleep_sec_until_5mins)

Loading data for 2020-03
Predicting 2020-03-03 00:00:00 nyctaxi-xgboost-endpoint
Sleep until next monitoring cycle. 3420 seconds

